<a href="https://colab.research.google.com/github/touseefyounas/Sankey-Stocks/blob/main/YahooFinanceAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
!pip install --upgrade plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 58.8 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.15.0
    Uninstalling plotly-5.15.0:
      Successfully uninstalled plotly-5.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [42]:
# Import requests to call the API; import pandas to convert response list/dict into Dataframes; import numpy to use array functions
import requests
import pandas as pd
import numpy as np
import plotly.graph_objects as go
#Stock ticker the statement is needed for and to be insterted in the URL
# stock='IBM'

# #url for the API requests call
# url = f"https://yahoo-finance15.p.rapidapi.com/api/yahoo/qu/quote/{stock}/income-statement"


# headers = {
# 	"X-RapidAPI-Key": "8cd9a70cc4mshde5422922530be0p16941bjsn4d9b78c54e6d",
# 	"X-RapidAPI-Host": "yahoo-finance15.p.rapidapi.com"
# }

# response = requests.get(url, headers=headers)

# #store the response json file in to the data
# data=response.json()

# #assign the annual income statement into the data value
# #data=data['incomeStatementHistory']['incomeStatementHistory']

#Stock ticker the statement is needed for and to be insterted in the URL
stock='IBM'

#url for the API requests call
url = 'https://mboum-finance.p.rapidapi.com/qu/quote/income-statement'

querystring = {'symbol':stock}

headers = {
	'X-RapidAPI-Key': '8cd9a70cc4mshde5422922530be0p16941bjsn4d9b78c54e6d',
	'X-RapidAPI-Host': 'mboum-finance.p.rapidapi.com'
}

response = requests.get(url, headers=headers, params=querystring)

#store the response json file in to the data
data=response.json()
data=data['body']['incomeStatementHistory']

In [50]:
fmt_values = []
for statement in data:
    statement_values = {}
    for key, value in statement.items():
        if isinstance(value, dict) and 'fmt' in value:
            statement_values[key] = value['fmt']
        elif isinstance(value, list):
            # Handle empty lists by setting them to None
            statement_values[key] = np.nan
    fmt_values.append(statement_values)

# Resulting list of income statement values
fmt_df=pd.DataFrame(fmt_values)

raw_values = []
for statement in data:
    statement_values = {}
    for key, value in statement.items():
        if isinstance(value, dict) and 'raw' in value:
            statement_values[key] = value['raw']
        elif isinstance(value, list):
            # Handle empty lists by setting them to None
            statement_values[key] = np.nan
    raw_values.append(statement_values)

raw_df=pd.DataFrame(raw_values)
raw_df

,endDate,totalRevenue,costOfRevenue,grossProfit,researchDevelopment,sellingGeneralAdministrative,nonRecurring,otherOperatingExpenses,totalOperatingExpenses,operatingIncome,...,incomeBeforeTax,incomeTaxExpense,minorityInterest,netIncomeFromContinuingOps,discontinuedOperations,extraordinaryItems,effectOfAccountingCharges,otherItems,netIncome,netIncomeApplicableToCommonShares
0,1688083200,15475000000,6974000000,8501000000,1687000000,4900000000,NaN,-248000000,13313000000,2162000000,...,2000000000,419000000,70000000,1581000000,2000000,NaN,NaN,NaN,1583000000,1583000000
1,1680220800,14251000000,6743000000,7508000000,1655000000,4858000000,NaN,-180000000,13076000000,1175000000,...,1058000000,124000000,68000000,934000000,-7000000,NaN,NaN,NaN,927000000,927000000
2,1672444800,16690000000,7058000000,9632000000,1604000000,4847000000,NaN,-232000000,13277000000,3413000000,...,3313000000,444000000,77000000,2869000000,-159000000,NaN,NaN,NaN,2710000000,2710000000
3,1664496000,14107000000,6677000000,7430000000,1611000000,4545000000,NaN,-120000000,12713000000,1394000000,...,-4501000000,-1287000000,71000000,-3214000000,18000000,NaN,NaN,NaN,-3196000000,-3196000000


In [44]:
sources=[]
targets=[]
values=[]
link_colors=[]

if raw_df['otherOperatingExpenses'].iloc[0] < 0 or pd.isna(raw_df['otherOperatingExpenses'].iloc[0]):

  if raw_df['incomeTaxExpense'].iloc[0] < 0:
     if pd.notna(raw_df['discontinuedOperations'].iloc[0]):
        if raw_df['discontinuedOperations'].iloc[0]<0:
          sources = [0,0,1,1,1,6,6,7,9,10,10]
          targets = [1,2,6,3,4,7,8,10,10,12,11]
          values = [raw_df.loc[0,'totalRevenue'],raw_df.loc[0,'grossProfit'],raw_df.loc[0,'costOfRevenue'],raw_df.loc[0,'sellingGeneralAdministrative'],
                    raw_df.loc[0,'researchDevelopment'],raw_df.loc[0,'operatingIncome'],raw_df.loc[0,'incomeBeforeTax'],abs(raw_df.loc[0,'totalOtherIncomeExpenseNet']),
                    abs(raw_df.loc[0,'incomeTaxExpense']),raw_df.loc[0,'netIncomeFromContinuingOps'],raw_df.loc[0,'netIncome'],abs(raw_df.loc[0,'discontinuedOperations'])]
          link_colors=['#A4CC9E','#D58A87','#A4CC9E','#D58A87','#D58A87','#A4CC9E','#D58A87','#A4CC9E','#A4CC9E','#A4CC9E','#D58A87'] # Colors Green:'#A4CC9E',Red: '#D58A87'
        if raw_df['discontinuedOperations'].iloc[0]>0:
          sources = [0,0,1,1,1,6,6,7,9,10,11]
          targets = [1,2,6,3,4,7,8,10,10,12,12]
          values = [raw_df.loc[0,'totalRevenue'],raw_df.loc[0,'grossProfit'],raw_df.loc[0,'costOfRevenue'],raw_df.loc[0,'sellingGeneralAdministrative'],
                    raw_df.loc[0,'researchDevelopment'],raw_df.loc[0,'operatingIncome'],raw_df.loc[0,'incomeBeforeTax'],abs(raw_df.loc[0,'totalOtherIncomeExpenseNet']),
                    abs(raw_df.loc[0,'incomeTaxExpense']),raw_df.loc[0,'netIncomeFromContinuingOps'],raw_df.loc[0,'netIncome'],raw_df.loc[0,'discontinuedOperations']]
          link_colors=['#A4CC9E','#D58A87','#A4CC9E','#D58A87','#D58A87','#A4CC9E','#D58A87','#A4CC9E','#A4CC9E','#A4CC9E','#A4CC9E'] #Colors Green:'#A4CC9E',Red: '#D58A87'
     if pd.isna(raw_df['discontinuedOperations'].iloc[0]):
        sources = [0,0,1,1,1,6,6,7,9]
        targets = [1,2,6,3,4,7,8,12,12]
        values = [raw_df.loc[0,'totalRevenue'],raw_df.loc[0,'grossProfit'],raw_df.loc[0,'costOfRevenue'],raw_df.loc[0,'sellingGeneralAdministrative'],
                  raw_df.loc[0,'researchDevelopment'],raw_df.loc[0,'operatingIncome'],raw_df.loc[0,'incomeBeforeTax'],abs(raw_df.loc[0,'totalOtherIncomeExpenseNet']),
                  abs(raw_df.loc[0,'incomeTaxExpense']),raw_df.loc[0,'netIncome']]
        link_colors=['#A4CC9E','#D58A87','#A4CC9E','#D58A87','#D58A87','#A4CC9E','#D58A87','#A4CC9E','#A4CC9E'] #Colors Green:'#A4CC9E',Red: '#D58A87'

  if raw_df['incomeTaxExpense'].iloc[0] > 0:
      if pd.notna(raw_df['discontinuedOperations'].iloc[0]):
        if raw_df['discontinuedOperations'].iloc[0]<0:
          sources = [0,0,1,1,1,6,6,7,7,10,10]
          targets = [1,2,6,3,4,7,8,10,9,12,11]
          values = [raw_df.loc[0,'totalRevenue'],raw_df.loc[0,'grossProfit'],raw_df.loc[0,'costOfRevenue'],raw_df.loc[0,'sellingGeneralAdministrative'],
                    raw_df.loc[0,'researchDevelopment'],raw_df.loc[0,'operatingIncome'],raw_df.loc[0,'incomeBeforeTax'],abs(raw_df.loc[0,'totalOtherIncomeExpenseNet']),
                    raw_df.loc[0,'incomeTaxExpense'],raw_df.loc[0,'netIncomeFromContinuingOps'],raw_df.loc[0,'netIncome'],abs(raw_df.loc[0,'discontinuedOperations'])]
          link_colors=['#A4CC9E','#D58A87','#A4CC9E','#D58A87','#D58A87','#A4CC9E','#D58A87','#A4CC9E','#D58A87','#A4CC9E','#D58A87'] #Colors Green:'#A4CC9E',Red: '#D58A87'
        if raw_df['discontinuedOperations'].iloc[0]>0:
          sources = [0,0,1,1,1,6,6,7,7,10,11]
          targets = [1,2,6,3,4,7,8,10,9,12,12]
          values = [raw_df.loc[0,'totalRevenue'],raw_df.loc[0,'grossProfit'],raw_df.loc[0,'costOfRevenue'],raw_df.loc[0,'sellingGeneralAdministrative'],
                    raw_df.loc[0,'researchDevelopment'],raw_df.loc[0,'operatingIncome'],raw_df.loc[0,'incomeBeforeTax'],abs(raw_df.loc[0,'totalOtherIncomeExpenseNet']),
                    raw_df.loc[0,'incomeTaxExpense'],raw_df.loc[0,'netIncomeFromContinuingOps'],raw_df.loc[0,'netIncome'],raw_df.loc[0,'discontinuedOperations']]
          link_colors=['#A4CC9E','#D58A87','#A4CC9E','#D58A87','#D58A87','#A4CC9E','#D58A87','#A4CC9E','#D58A87','#A4CC9E','#A4CC9E'] #Colors Green:'#A4CC9E',Red: '#D58A87'

      if pd.isna(raw_df['discontinuedOperations'].iloc[0]):
          sources = [0,0,1,1,1,6,6,7,7]
          targets = [1,2,6,3,4,7,8,12,9]
          values = [raw_df.loc[0,'totalRevenue'],raw_df.loc[0,'grossProfit'],raw_df.loc[0,'costOfRevenue'],raw_df.loc[0,'sellingGeneralAdministrative'],
                    raw_df.loc[0,'researchDevelopment'],raw_df.loc[0,'operatingIncome'],raw_df.loc[0,'incomeBeforeTax'],abs(raw_df.loc[0,'totalOtherIncomeExpenseNet']),
                    raw_df.loc[0,'incomeTaxExpense'],raw_df.loc[0,'netIncome']]
          link_colors=['#A4CC9E','#D58A87','#A4CC9E','#D58A87','#D58A87','#A4CC9E','#D58A87','#A4CC9E','#D58A87'] #Colors - Green:'#A4CC9E',Red: '#D58A87'

if raw_df['otherOperatingExpenses'].iloc[0] > 0 and pd.notna(raw_df['otherOperatingExpenses'].iloc[0]):
  if raw_df['incomeTaxExpense'].iloc[0] < 0:
     if pd.notna(raw_df['discontinuedOperations'].iloc[0]):
        if raw_df['discontinuedOperations'].iloc[0]<0:
          sources = [0,0,1,1,1,1,6,6,7,9,10,10]
          targets = [1,2,6,3,4,5,7,8,10,10,12,11]
          values = [raw_df.loc[0,'totalRevenue'],raw_df.loc[0,'grossProfit'],raw_df.loc[0,'costOfRevenue'],raw_df.loc[0,'sellingGeneralAdministrative'],
                    raw_df.loc[0,'researchDevelopment'],raw_df.loc[0,'otherOperatingExpenses'],raw_df.loc[0,'operatingIncome'],raw_df.loc[0,'incomeBeforeTax'],abs(raw_df.loc[0,'totalOtherIncomeExpenseNet']),
                    abs(raw_df.loc[0,'incomeTaxExpense']),raw_df.loc[0,'netIncomeFromContinuingOps'],raw_df.loc[0,'netIncome'],abs(raw_df.loc[0,'discontinuedOperations'])]
          link_colors=['#A4CC9E','#D58A87','#A4CC9E','#D58A87','#D58A87','#D58A87','#A4CC9E','#D58A87','#A4CC9E','#A4CC9E','#A4CC9E','#D58A87'] # Colors Green:'#A4CC9E',Red: '#D58A87'
        if raw_df['discontinuedOperations'].iloc[0]>0:
          sources = [0,0,1,1,1,1,6,6,7,9,10,11]
          targets = [1,2,6,3,4,5,7,8,10,10,12,12]
          values = [raw_df.loc[0,'totalRevenue'],raw_df.loc[0,'grossProfit'],raw_df.loc[0,'costOfRevenue'],raw_df.loc[0,'sellingGeneralAdministrative'],raw_df.loc[0,'researchDevelopment'],
                    raw_df.loc[0,'otherOperatingExpenses'],raw_df.loc[0,'operatingIncome'],raw_df.loc[0,'incomeBeforeTax'],abs(raw_df.loc[0,'totalOtherIncomeExpenseNet']),
                    abs(raw_df.loc[0,'incomeTaxExpense']),raw_df.loc[0,'netIncomeFromContinuingOps'],raw_df.loc[0,'netIncome'],raw_df.loc[0,'discontinuedOperations']]
          link_colors=['#A4CC9E','#D58A87','#A4CC9E','#D58A87','#D58A87','#D58A87','#A4CC9E','#D58A87','#A4CC9E','#A4CC9E','#A4CC9E','#A4CC9E'] #Colors Green:'#A4CC9E',Red: '#D58A87'
     if pd.isna(raw_df['discontinuedOperations'].iloc[0]):
        sources = [0,0,1,1,1,1,6,6,7,9]
        targets = [1,2,6,3,4,5,7,8,12,12]
        values = [raw_df.loc[0,'totalRevenue'],raw_df.loc[0,'grossProfit'],raw_df.loc[0,'costOfRevenue'],raw_df.loc[0,'sellingGeneralAdministrative'],
                  raw_df.loc[0,'researchDevelopment'],raw_df.loc[0,'otherOperatingExpenses'],raw_df.loc[0,'operatingIncome'],raw_df.loc[0,'incomeBeforeTax'],
                  abs(raw_df.loc[0,'totalOtherIncomeExpenseNet']),abs(raw_df.loc[0,'incomeTaxExpense']),raw_df.loc[0,'netIncome']]
        link_colors=['#A4CC9E','#D58A87','#A4CC9E','#D58A87','#D58A87','#D58A87','#A4CC9E','#D58A87','#A4CC9E','#A4CC9E'] #Colors Green:'#A4CC9E',Red: '#D58A87'

  if raw_df['incomeTaxExpense'].iloc[0] > 0:
      if pd.notna(raw_df['discontinuedOperations'].iloc[0]):
        if raw_df['discontinuedOperations'].iloc[0]<0:
          sources = [0,0,1,1,1,1,6,6,7,7,10,10]
          targets = [1,2,6,3,4,5,7,8,10,9,12,11]
          values = [raw_df.loc[0,'totalRevenue'],raw_df.loc[0,'grossProfit'],raw_df.loc[0,'costOfRevenue'],raw_df.loc[0,'sellingGeneralAdministrative'],raw_df.loc[0,'researchDevelopment'],
                    raw_df.loc[0,'otherOperatingExpenses'],raw_df.loc[0,'operatingIncome'],raw_df.loc[0,'incomeBeforeTax'],abs(raw_df.loc[0,'totalOtherIncomeExpenseNet']),
                    raw_df.loc[0,'incomeTaxExpense'],raw_df.loc[0,'netIncomeFromContinuingOps'],raw_df.loc[0,'netIncome'],abs(raw_df.loc[0,'discontinuedOperations'])]
          link_colors=['#A4CC9E','#D58A87','#A4CC9E','#D58A87','#D58A87','#D58A87','#A4CC9E','#D58A87','#A4CC9E','#D58A87','#A4CC9E','#D58A87'] #Colors Green:'#A4CC9E',Red: '#D58A87'
        if raw_df['discontinuedOperations'].iloc[0]>0:
          sources = [0,0,1,1,1,1,6,6,7,7,10,11]
          targets = [1,2,6,3,4,5,7,8,10,9,12,12]
          values = [raw_df.loc[0,'totalRevenue'],raw_df.loc[0,'grossProfit'],raw_df.loc[0,'costOfRevenue'],raw_df.loc[0,'sellingGeneralAdministrative'],raw_df.loc[0,'researchDevelopment'],
                    raw_df.loc[0,'otherOperatingExpenses'],raw_df.loc[0,'operatingIncome'],raw_df.loc[0,'incomeBeforeTax'],abs(raw_df.loc[0,'totalOtherIncomeExpenseNet']),
                    raw_df.loc[0,'incomeTaxExpense'],raw_df.loc[0,'netIncomeFromContinuingOps'],raw_df.loc[0,'netIncome'],raw_df.loc[0,'discontinuedOperations']]
          ink_colors=[['#A4CC9E','#D58A87','#A4CC9E','#D58A87','#D58A87','#D58A87','#A4CC9E','#D58A87','#A4CC9E','#D58A87','#A4CC9E','#A4CC9E']] #Colors Green:'#A4CC9E',Red: '#D58A87'
      if pd.isna(raw_df['discontinuedOperations'].iloc[0]):
          sources = [0,0,1,1,1,1,6,6,7,7]
          targets = [1,2,6,3,4,5,7,8,12,9]
          values = [raw_df.loc[0,'totalRevenue'],raw_df.loc[0,'grossProfit'],raw_df.loc[0,'costOfRevenue'],raw_df.loc[0,'sellingGeneralAdministrative'],
                    raw_df.loc[0,'researchDevelopment'],raw_df.loc[0,'otherOperatingExpenses'],raw_df.loc[0,'operatingIncome'],raw_df.loc[0,'incomeBeforeTax'],
                    abs(raw_df.loc[0,'totalOtherIncomeExpenseNet']),raw_df.loc[0,'incomeTaxExpense'],raw_df.loc[0,'netIncome']]
          link_colors=['#A4CC9E','#D58A87','#A4CC9E','#D58A87','#D58A87','#D58A87','#A4CC9E','#D58A87','#A4CC9E','#D58A87'] #Colors - Green:'#A4CC9E',Red: '#D58A87'

In [4]:
latest_raw=raw_df.loc[0] #Over here you can add a list of drop down options with an if statement to iterate for the selected year e.g. 2022= [0], 2021=[1] etc.
latest_raw=pd.DataFrame(latest_raw)
latest_raw

,0
endDate,1.688083e+09
totalRevenue,1.547500e+10
costOfRevenue,6.974000e+09
grossProfit,8.501000e+09
researchDevelopment,1.687000e+09
sellingGeneralAdministrative,4.900000e+09
nonRecurring,NaN
otherOperatingExpenses,-2.480000e+08
totalOperatingExpenses,1.331300e+10
operatingIncome,2.162000e+09


In [5]:
fmt_df

,endDate,totalRevenue,costOfRevenue,grossProfit,researchDevelopment,sellingGeneralAdministrative,nonRecurring,otherOperatingExpenses,totalOperatingExpenses,operatingIncome,...,incomeBeforeTax,incomeTaxExpense,minorityInterest,netIncomeFromContinuingOps,discontinuedOperations,extraordinaryItems,effectOfAccountingCharges,otherItems,netIncome,netIncomeApplicableToCommonShares
0,2023-06-30,15.47B,6.97B,8.5B,1.69B,4.9B,NaN,-248M,13.31B,2.16B,...,2B,419M,70M,1.58B,2M,NaN,NaN,NaN,1.58B,1.58B
1,2023-03-31,14.25B,6.74B,7.51B,1.66B,4.86B,NaN,-180M,13.08B,1.18B,...,1.06B,124M,68M,934M,-7M,NaN,NaN,NaN,927M,927M
2,2022-12-31,16.69B,7.06B,9.63B,1.6B,4.85B,NaN,-232M,13.28B,3.41B,...,3.31B,444M,77M,2.87B,-159M,NaN,NaN,NaN,2.71B,2.71B
3,2022-09-30,14.11B,6.68B,7.43B,1.61B,4.54B,NaN,-120M,12.71B,1.39B,...,-4.5B,-1.29B,71M,-3.21B,18M,NaN,NaN,NaN,-3.2B,-3.2B


In [7]:
fig = go.Figure(data=[go.Sankey(
    arrangement="freeform",
    node = dict(
      pad = 30,
      thickness = 20,
      line = dict(color = "white", width =0),
      label = [f"Total Revenue ${fmt_df.at[0,'totalRevenue']}",                                  #0
              f"Gross Profit ${fmt_df.at[0,'grossProfit']}",                                     #1
              f"Cost of Revenue -${fmt_df.at[0,'costOfRevenue']}",                               #2
              f"SG&A -${fmt_df.at[0,'sellingGeneralAdministrative']}",                           #3
              f"R&D -${fmt_df.at[0,'researchDevelopment']}",                                     #4
              f"Other -${fmt_df.at[0,'otherOperatingExpenses']}",                                #5
              f"Operating Income ${fmt_df.at[0,'operatingIncome']}",                             #6
              f"Income Before Tax ${fmt_df.at[0,'incomeBeforeTax']}",                            #7
              f"Other Income Expenses ${fmt_df.at[0,'totalOtherIncomeExpenseNet']}",             #8
              f"Income Tax ${fmt_df.at[0,'incomeTaxExpense']}",                                  #9
              f"Net Income - Cont Ops ${fmt_df.at[0,'netIncomeFromContinuingOps']}",             #10
              f"Discontinued Operations ${fmt_df.at[0,'discontinuedOperations']}",               #11
              f"Net Income ${fmt_df.at[0,'netIncome']}"                                          #12
                ],
      color = ['#519E3F','#519E3F','#BC271B','#BC271B','#BC271B','#BC271B','#519E3F','#519E3F','#BC271B','#BC271B','#519E3F','#519E3F','#519E3F']
    ),
    textfont=dict(family='sans serif',size=20,
                  color='rgba(0,0,0,1)'),
    link = dict(
      source = sources,
      target = targets,
      value = values,
      color = link_colors
))])

# add title to the diagram
fig.update_layout(title={'text':'IBM 2022  Income Statement','y':0.95,'x':0.5,'xanchor':'center','yanchor':'top','font_size':30},yaxis={'categoryorder':'total ascending'},paper_bgcolor='rgb(248,248,255)',
    plot_bgcolor='rgb(248,248,255)',)


fig.show()

In [ ]:
income_year=fmt_df
income_year['year'] = fmt_df['endDate'].str[:4]
income_year

,endDate,totalRevenue,costOfRevenue,grossProfit,researchDevelopment,sellingGeneralAdministrative,nonRecurring,otherOperatingExpenses,totalOperatingExpenses,operatingIncome,...,incomeTaxExpense,minorityInterest,netIncomeFromContinuingOps,discontinuedOperations,extraordinaryItems,effectOfAccountingCharges,otherItems,netIncome,netIncomeApplicableToCommonShares,year
0,2022-12-31,60.53B,27.84B,32.69B,6.57B,19.2B,NaN,-642M,52.96B,7.57B,...,-626M,77M,1.78B,-143M,NaN,NaN,NaN,1.64B,1.64B,2022
1,2021-12-31,57.35B,25.86B,31.49B,6.49B,19.8B,NaN,-577M,51.58B,5.77B,...,124M,95M,4.71B,1.03B,NaN,NaN,NaN,5.74B,5.74B,2021
2,2020-12-31,55.18B,24.31B,30.86B,6.26B,20.56B,NaN,-620M,50.52B,4.66B,...,-1.36B,129M,3.93B,1.66B,NaN,NaN,NaN,5.59B,5.59B,2020
3,2019-12-31,57.71B,26.18B,31.53B,5.91B,18.72B,NaN,-639M,50.17B,7.54B,...,60M,144M,7.15B,2.29B,NaN,NaN,NaN,9.43B,9.43B,2019


In [45]:
node_x = [0,    # Total Revenue
          0.2,  # Gross Profit
          0.2,  # Cost of Revenue
          0.35,  # SG&A
          0.35,  # R&D
          0.9,  # Other Operating Expenses
          0.4,  # Operating Income
          0.6,  # Income Before Tax
          0.6,  # Other Income Expenses
          0.8,  # Income Tax
          0.5,  # Net Income - Cont Ops
          1.0,  # Discontinued Operations
          0.9]  # Net Income

node_y = [0.5,  # Total Revenue
          0.2,  # Gross Profit
          0.8,  # Cost of Revenue
          0.55,  # SG&A
          0.7,  # R&D
          0.6,  # Other Operating Expenses
          0.3,  # Operating Income
          0.2,  # Income Before Tax
          0.7,  # Other Income Expenses
          0.8,  # Income Tax
          0.7,  # Net Income - Cont Ops
          0.8,  # Discontinued Operations
          0.3]  # Net Income


In [49]:
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 30,
      thickness = 20,
      line = dict(color = "white", width =0),
      label = [f"Total Revenue ${fmt_df.at[0,'totalRevenue']}",                                  #0
              f"Gross Profit ${fmt_df.at[0,'grossProfit']}",                                     #1
              f"Cost of Revenue -${fmt_df.at[0,'costOfRevenue']}",                               #2
              f"SG&A -${fmt_df.at[0,'sellingGeneralAdministrative']}",                           #3
              f"R&D -${fmt_df.at[0,'researchDevelopment']}",                                     #4
              f"Other -${fmt_df.at[0,'otherOperatingExpenses']}",                                #5
              f"Operating Income ${fmt_df.at[0,'operatingIncome']}",                             #6
              f"Income Before Tax ${fmt_df.at[0,'incomeBeforeTax']}",                            #7
              f"Other Income Expenses ${fmt_df.at[0,'totalOtherIncomeExpenseNet']}",             #8
              f"Income Tax ${fmt_df.at[0,'incomeTaxExpense']}",                                  #9
              f"Net Income - Cont Ops ${fmt_df.at[0,'netIncomeFromContinuingOps']}",             #10
              f"Discontinued Operations ${fmt_df.at[0,'discontinuedOperations']}",               #11
              f"Net Income ${fmt_df.at[0,'netIncome']}"                                          #12
                ],
      color = ['#519E3F','#519E3F','#BC271B','#BC271B','#BC271B','#BC271B','#519E3F','#519E3F','#BC271B','#BC271B','#519E3F','#519E3F','#519E3F'],
      # x=node_x,
      # y=node_y,
    ),
    textfont=dict(family='sans serif',size=20,
                  color='rgba(0,0,0,1)'),
    link = dict(
      source = sources,
      target = targets,
      value = values,
      color = link_colors
))])

# add title to the diagram
fig.update_layout(title={'text':f'{stock} 2022  Income Statement','y':0.95,'x':0.5,'xanchor':'center','yanchor':'top','font_size':30},yaxis={'categoryorder':'total ascending'},paper_bgcolor='rgb(248,248,255)',
    plot_bgcolor='rgb(248,248,255)',)


fig.show()